In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv2
import tensorflow as tf
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Bidirectional, LSTM, Dense, Dropout, Flatten, Embedding,Reshape
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from keras import layers
import warnings

warnings.filterwarnings('ignore')

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
img_dir = '/kaggle/input/persianocrdataset/Shotor_Images'
df=pd.read_csv('/kaggle/input/persianocrdataset/Shotor_Words.csv')
label=df['word']

In [ ]:
target_size=(100,100)
images=[]
for i in range(len(df)):
    img_dir = '/kaggle/input/persianocrdataset/Shotor_Images/Shotor_Images/' + df.loc[i, 'image']
    image = cv2.imread(img_dir, cv2.IMREAD_GRAYSCALE)
    img_resized = cv2.resize(image, target_size)
    kernel = np.ones((2, 2), np.uint8)
    dilated_image = cv2.dilate(img_resized, kernel, iterations=1)
    images.append(dilated_image)

In [5]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
la = label_encoder.fit_transform(label)

In [9]:
la[:20]

array([14815, 17872, 26883, 64988, 53628, 35507, 10832, 63825, 10832,
       65738, 53628, 29429,     6, 26883, 64988, 26579, 25348, 65860,
       13120, 22075])

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
images = np.array(images)
images = images / 255.0

x_train, x_test, y_train, y_test = train_test_split(images, la, test_size=0.2, random_state=42, shuffle=False)

datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

batch_size = 32

x_train_reshaped = x_train.reshape((-1,) + target_size + (1,))
x_test_reshaped = x_test.reshape((-1,) + target_size + (1,))

del x_train
del x_test




In [7]:
train_generator = datagen.flow(x_train_reshaped, y_train, batch_size=batch_size, shuffle=False)
test_generator = datagen.flow(x_test_reshaped, y_test, batch_size=batch_size, shuffle=False)

In [8]:
del x_train_reshaped
del x_test_reshaped
del y_test


In [9]:

model = models.Sequential()

# CNN layers
model.add(layers.Conv2D(32, (3, 3), activation="relu", input_shape=(100, 100, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation="relu"))
model.add(layers.MaxPooling2D((2, 2)))


In [10]:
model.add(layers.Reshape((-1, 64)))

# BiLSTM layers
model.add(layers.Bidirectional(layers.LSTM(128, return_sequences=True)))
model.add(layers.Bidirectional(layers.LSTM(64, return_sequences=True)))
model.add(layers.Bidirectional(layers.LSTM(32, return_sequences=True)))


model.add(layers.Flatten())

# Dense layers
model.add(layers.Dense(64, activation="relu"))
model.add(layers.Dense(len(np.unique(y_train)), activation="softmax"))  # Adjust based on the number of classes



In [11]:
from tensorflow.keras.optimizers.legacy import Adam
optimizer = Adam(lr=0.001)  # You can adjust the learning rate
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 98, 98, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 49, 49, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 47, 47, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 23, 23, 64)        0         
 g2D)                                                            
                                                                 
 reshape (Reshape)           (None, 529, 64)           0         
                                                                 
 bidirectional (Bidirection  (None, 529, 256)          1

In [12]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
history = model.fit(train_generator, epochs=500, batch_size=32, validation_data=test_generator, callbacks=[early_stopping])

Epoch 1/500
3000/3000 [==============================] - 528s 171ms/step - loss: nan - accuracy: 2.0833e-05 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 2/500
3000/3000 [==============================] - 511s 170ms/step - loss: nan - accuracy: 2.0833e-05 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 3/500
3000/3000 [==============================] - 511s 170ms/step - loss: nan - accuracy: 2.0833e-05 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 4/500
3000/3000 [==============================] - 512s 171ms/step - loss: nan - accuracy: 2.0833e-05 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 5/500
3000/3000 [==============================] - 512s 171ms/step - loss: nan - accuracy: 2.0833e-05 - val_loss: nan - val_accuracy: 0.0000e+00


In [13]:
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc}")


750/750 [==============================] - 59s 78ms/step - loss: nan - accuracy: 0.0000e+00
Test Accuracy: 0.0
